<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/Keras_30_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install lightning einops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 22.3 MB/s eta 0:00:00


In [ ]:
import torch, torchvision
from torch import nn

In [ ]:
from torchvision.transforms import v2 as T

In [ ]:
import pytorch_lightning as ptl

In [ ]:
import torchmetrics

In [ ]:
from einops.layers.torch import Rearrange

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
train_transforms = T.Compose([
    T.RandomRotation(10),
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True)
])

In [ ]:
test_transforms = T.Compose([
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True)
])

In [ ]:
train_data = torchvision.datasets.CIFAR10('/cifar10/', download=True, transform=train_transforms)
test_data = torchvision.datasets.CIFAR10('/cifar10/', download=True, transform=test_transforms)

100%|██████████| 170498071/170498071 [00:04<00:00, 38868204.47it/s]


Extracting /cifar10/cifar-10-python.tar.gz to /cifar10/
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
from einops import rearrange

In [ ]:
class InputEmbedding(nn.Module):
    def __init__(self, patch_size=4, input_shape=32, embedding_dim=32):
        super().__init__()
        self.patch_size = patch_size
        self.embedding_dim = embedding_dim
        num_patches = (input_shape // self.patch_size) * (input_shape // self.patch_size)
        self.patchifier = Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size)
        self.proj = nn.Linear(self.patch_size*self.patch_size*3, self.embedding_dim)
        self.pos_embeddings = nn.Parameter(torch.randn((1, num_patches  + 1, self.embedding_dim)))
        self.cls_token = nn.Parameter(torch.randn((1, 1, self.embedding_dim)))

    def forward(self, x):
        x = self.proj(self.patchifier(x))
        cls_token = self.cls_token.repeat(x.shape[0], 1, 1)
        x = torch.concat([cls_token, x], axis=1)
        return x + self.pos_embeddings

In [ ]:
class RecurrentMHA(nn.Module):
  def __init__(self, num_heads=4, embedding_dim=32, n_latents=8):
    super().__init__()
    self.Q = nn.Parameter(torch.randn((1, n_latents, embedding_dim)))
    self.keys = nn.Linear(embedding_dim, embedding_dim)
    self.values = nn.Linear(embedding_dim, embedding_dim)
    self.mha = nn.MultiheadAttention(embedding_dim, num_heads, batch_first=True)

  def forward(self, x):
    K = self.keys(x)
    V = self.values(x)
    Q_t = self.Q.repeat(x.shape[0], 1, 1)
    y = torch.zeros_like(x, device=x.device)
    for i in range(x.shape[1]):
      Q_t = self.mha(Q_t, K[:, i:i+1], V[:, i:i+1], need_weights=False)[0]
      y[:, i] = Q_t[:, 0]
    return y

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim=32):
      super().__init__()
      self.mha = RecurrentMHA()
      self.mlp = nn.Sequential(
          nn.Linear(embedding_dim, embedding_dim * 3),
          nn.GELU(),
          nn.Linear(embedding_dim*3, embedding_dim)
      )
      self.ln1 = nn.LayerNorm(embedding_dim)
      self.ln2 = nn.LayerNorm(embedding_dim)

    def forward(self, x):
      h = self.mha(self.ln1(x)) + x
      h = self.mlp(self.ln1(x)) + x
      return x

In [ ]:
class VisionTransformer(nn.Module):
  def __init__(self, embedding_dim=32):
    super().__init__()
    self.inp_embedding = InputEmbedding()
    self.block1 = TransformerBlock()
    self.block2 = TransformerBlock()
    self.block3 = TransformerBlock()
    self.classifier = nn.Linear(embedding_dim, 10)

  def forward(self, x):
    x = self.inp_embedding(x)
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    return self.classifier(x[:, 0, :])

In [ ]:
vit = VisionTransformer()

In [ ]:
class PTLWrapper(ptl.LightningModule):
    def __init__(self, cnn):
      super().__init__()
      self.cnn = cnn
      self.acc = torchmetrics.Accuracy(task='multiclass', num_classes=10)
      self.loss = torch.nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
      xb, yb = batch
      ypred = self.cnn(xb)
      loss = self.loss(ypred, yb)
      self.log("train_loss", loss, prog_bar=True, logger=False)
      return loss

    def validation_step(self, batch, batch_idx):
      xb, yb = batch
      ypred = self.cnn(xb)
      self.acc(ypred, yb)
      self.log('val_accuracy', self.acc, on_step=False, on_epoch=True, prog_bar=True, logger=False)

    def configure_optimizers(self):
      return torch.optim.Adam(self.cnn.parameters())

In [ ]:
#cnn = torch.compile(VisionTransformer())
cnn = VisionTransformer()
ptl_wrapper = PTLWrapper(cnn)

In [ ]:
trainer = ptl.Trainer(accelerator='gpu', devices=1,  max_epochs=5)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(ptl_wrapper, train_dataloaders=train_loader,
            val_dataloaders=test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type               | Params
--------------------------------------------
0 | cnn  | VisionTransformer  | 43.0 K
1 | acc  | MulticlassAccuracy | 0     
2 | loss | CrossEntropyLoss   | 0     
--------------------------------------------
43.0 K    Trainable params
0         Non-trainable params
43.0 K    Total params
0.172     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
